# Sailing RL Challenge - Training on Colab GPU

This notebook allows you to train your DQN agent on Google Colab's free GPU.

In [ ]:
# Cell 1: Clone repository and install
!git clone https://github.com/YOUR_USERNAME/sailing-rl-challenge.git
%cd sailing-rl-challenge
!pip install -q -e .

In [ ]:
# Cell 2: Mount Google Drive for checkpoints
from google.colab import drive
drive.mount('/content/drive')

# Create checkpoint directory in Drive
!mkdir -p /content/drive/MyDrive/sailing_checkpoints
!mkdir -p /content/drive/MyDrive/sailing_submissions

In [ ]:
# Cell 3: Verify GPU availability
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# Cell 4: Collect normalization stats (RUN ONCE)
!python src/training/train_dqn.py \
    --collect-stats \
    --config config/dqn_baseline.yaml

In [ ]:
# Cell 5: Train DQN with GPU
!python src/training/train_dqn.py \
    --config config/dqn_baseline.yaml \
    --device cuda

In [ ]:
# Cell 6: Copy checkpoints and submissions to Drive
!cp -r checkpoints/* /content/drive/MyDrive/sailing_checkpoints/
!cp -r src/submission/* /content/drive/MyDrive/sailing_submissions/

print("✅ Files saved to Google Drive")

In [ ]:
# Cell 7: Generate submission file from best checkpoint
from src.agents.my_agent_DQN import DQNTrainer
from src.utils.save_my_dqn import save_dqn_agent
from wind_scenarios.env_sailing import SailingEnv
from wind_scenarios import get_wind_scenario
import torch

# Load checkpoint
env = SailingEnv(**get_wind_scenario('training_1'))
trainer = DQNTrainer(env, stats_path='data/normalization_stats.pkl', device='cuda')

checkpoint = torch.load('checkpoints/dqn/final_model.pth')
trainer.q_network.load_state_dict(checkpoint)

# Generate submission
output_path = '/content/drive/MyDrive/sailing_submissions/my_agent_final.py'
save_dqn_agent(trainer, output_path)

print(f"✅ Submission file saved to {output_path}")